## Positive Dimensional Local minimizers

In [ ]:
# Globtim Notebook Setup - Universal Header Cell
# This cell automatically detects your environment and sets up the appropriate configuration
# No editing required - works from any location in the project

include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

In [ ]:
# Constants and Parameters
const n, a, b = 2, 1, 1
f(x) = (x[1]^2+ x[2]^2 -1)^2 # Objective function

In [ ]:
d = 8 # Initial Degree 
SMPL = 100 # Number of samples
center = [0.0, 0.0]
TR = TestInput(f,
                dim=n,
                center=[0.0, 0.0],
                GN=SMPL,
                sample_range=[1.2, 1.5]
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision= AdaptivePrecision)
pol_lege = Constructor(TR, d, basis=:legendre);


In [ ]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=false,
    power_of_two_denom=pol_cheb.power_of_two_denom
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=true)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR)

In [ ]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.001)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.001);

In [ ]:
using CairoMakie
CairoMakie.activate!()

In [ ]:
fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb, chebyshev_levels=true, figure_size=(800, 800))

In [ ]:
fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true, figure_size = (800, 800))

In [ ]:
# Commented out save command
# save("Deuflhard_w_22.pdf", fig_1)

In [ ]:
using StaticArrays
Lambda = SupportGen(n, pol_cheb.degree)
V = Globtim.lambda_vandermonde(Lambda, pol_cheb.grid, basis=pol_cheb.basis)
# Evaluate polynomial at grid points: polynomial_values = V * coefficients  
polynomial_values = V * pol_cheb.coeffs

# Now use the existing plotting infrastructure but replace pol_cheb.z with polynomial_values
coords = transform_coordinates(pol_cheb.scale_factor, pol_cheb.grid, TR.center)

# Create contour plot
x_unique = sort(unique(coords[:, 1]))
y_unique = sort(unique(coords[:, 2]))
Z = fill(NaN, (length(y_unique), length(x_unique)))

for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], polynomial_values))
    i = findfirst(≈(y), y_unique)
    j = findfirst(≈(x), x_unique)
    if !isnothing(i) && !isnothing(j)
        Z[i, j] = z
    end
end

# Find the minimum value in your polynomial
min_val = minimum(polynomial_values)
max_val = maximum(polynomial_values)

# Create logarithmically spaced levels close to minimum
if min_val > 0
    # If minimum is positive, use log spacing from min to max
    levels = exp.(range(log(min_val), log(max_val), length=40))
else
    # If minimum is near zero or negative, focus levels near zero
    abs_min = abs(min_val)
    levels_pos = exp.(range(log(1e-10), log(max_val), length=15))
    levels_neg = -exp.(range(log(1e-10), log(abs_min), length=5))
    levels = sort([levels_neg; levels_pos])
end

# Create figure and axis
fig = Figure(size=(800, 800))
ax = Axis(fig[1, 1], xlabel="x1", ylabel="x2", title="Polynomial Approximation with Custom Levels")

# Plot with custom levels
contour!(ax, x_unique, y_unique, Z, levels=levels, linewidth=2)
scatter!(ax, df_cheb.x1, df_cheb.x2, color=:red, markersize=10, label="Critical Points")
scatter!(ax, df_min_cheb.x1, df_min_cheb.x2, color=:blue, markersize=12, marker=:star5, label="Minimizers")
axislegend(ax)
fig